# Design Considerations

## Part 1: Identify FIVE design decisions for this system.
Argue for how this design decision is significant to the system, and your plans to analyze this design. If needed, use the dataset provided to you.

### 1. Search Technique
Choosing a search technique is an important design consideration for the retrieval service in the system. The search module is responsible for searching the database of known embeddings to find the most similar samples to the input probe. This search needs to be accurate so that we don't misidentify an intruder as a known employee (i.e. a false positive), but also needs to be efficient so that entrants don't have to wait at the access point for long periods of time for the system to complete its search. Even if we decide that we'll use K-Nearest Neighbors for our search algorithm, there are still two main aspects of the search module to consider: exact vs. approximate search and indexing strategy. Implementing some kind of indexing is important because it allows the data to be organized in a way that makes searching, inserting, deleting, and retrieval more efficient.
- **Exact vs. Approximate:** Exact searches like brute force can be highly accurate and can guarantee that the returned samples are the most similar to the input probe; however, this can be computationally expensive, especially if the embeddings database has many samples or high dimensionality. Approximate searches don't require visiting every single sample in the embeddings database, and therefore are more computationally efficient; however, may be less accurate since they don't guarantee that the optimal solution is identified, since they only search for an adequate solution that's good enough.
- **Indexing Strategies:** Indexing strategies organize the data in a way the improves efficiency for search and retrieval. Some strategies like KD trees can be applied to both exact and approximate searches, while others like local sensitivity hashing (LSH) and hierarchical navigable small world graphs (HNSW) are intended for approximate searches. KD trees partition the data by dimension into "hyperrectangles", allowing for more efficient search since some branches can be eliminated; however, tree depth increases with dimensionality, making this less efficient for high dimensional data. LSH groups similar items into buckets by increasing the probability of hashing collisions for similar items, and is a good alternative for high dimensional data; however, may result in false negatives due to its probablistic nature. HNSW is an accurate, scalable and efficient solution for high dimensional data, where the data is organized into heirarchical layers, with each course layers first and more fine-grained layers towards the bottom; however, storing a graph of all of the layers can be memory intensive.

**Analysis Plan**

To analyze the different search techniques, each variation (brute force, exact KNN,KD trees, LSH, HNSW) can be run on a validation set of known embedding vectors that have been precomputed from the gallery database. The mean average precision can then be calculated for each run, so that the quality of the search outputs (ranked nearest samples) can be compared for each search technique. Additionally, the mean search time per sample will be tracked, so we can determine which technique is the most efficient. The search technique resulting in the best balance of mean average precision and efficiency would be chosen for deployment. Mean average precision is chosen as a comparison metric since it takes into account both the number of positive samples identified, as well as the rank of those samples, unlike precision@k, recall@k, and mean reciprocal rank.

### 2. Similarity Measure
Another important design consideration is the similarity measure used in the Nearest Neighbor search when calculating the distance between two embedding vectors. The distance metric must ensure that higher distances are associated with dissimilar images and lower distances are associated with similar images. Different distance metrics can lead to different results for the same points, so it's important to carefully choose the distance measure. Some popular distance metrics include Manhattan distance (aka "city block" distance), Euclidean distance (aka "straight line" distance), Minkowski distance (a generalized version of Manhattan and Euclidean distances), and cosine similarity which measures the angle between two vectors.

**Analysis Plan**

To analyze the different similarity measures, each variation (Manhattan distance, Euclidean distance, Minkwoski distance with different values for the p parameter, and cosine similarity) can be run on a validation set of known embedding vectors that have been precomputed from the gallery database. The mean average precision can then be calculated for each run, so that the quality of the search outputs (ranked nearest samples) can be compared for the search using each similarity measure. Additionally, the mean search time per sample will be tracked, so we can identify any differences in efficiency between the similarity measures. The similarity measure resulting in the best balance of mean average precision and efficiency would be chosen for deployment.

### 3.
TBD

**Analysis Plan**

TBD

### 4. 
TBD

**Analysis Plan**

TBD

### 5.
TBD

**Analysis Plan**

TBD

### References
- Pham, A. (2022, April 20). A look into K-Dimensional Trees. Medium. https://medium.com/smucs/a-look-into-k-dimensional-trees-290ec69dffe9
- MyScale. (2023, June 14). HNSW vs KNN: The Ultimate Algorithm Showdown. https://myscale.com/blog/hnsw-vs-knn-algorithm-comparison/
- Ng, A. (2018, April 16). kNN.16 Locality sensitive hashing (LSH) [Video]. YouTube. https://www.youtube.com/watch?v=LqcwaW2YE_c
- Kaushik, S. (2021, January 11). Importance of Distance Metrics in Machine Learning Modelling. Towards Data Science. https://towardsdatascience.com/importance-of-distance-metrics-in-machine-learning-modelling-e51395ffe60d
- Gokte, S. A. (2020, November 17). Most Popular Distance Metrics Used in KNN and When to Use Them. KDnuggets. https://www.kdnuggets.com/2020/11/most-popular-distance-metrics-knn.html

## Part 2: Analyze TWO out of the five design decisions. Add the results of your analysis.

### 1. TBD

In [ ]:
# Analysis 1

### 2. TBD

In [ ]:
# Analysis 2